In [1]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import TimeoutException
from time import sleep
import pandas as pd

# Testing Part

In [29]:
#Testing on Selenium
driver = webdriver.Chrome(executable_path="D:\\chromedriver_win32\\chromedriver.exe")
driver.maximize_window()
#driver.implicity_wait(10)

driver.get("https://google.co.in")
driver.find_element_by_name('q').send_keys('Selenium')
driver.find_element_by_name('btnK').submit()
driver.close()


# Web Scrapping on Shopee product- Ahmad Jasmine Green Tea ( 25 tea bags)

In [2]:
item_cost, item_init_cost, item_loc = [],[],[]
item_name, items_sold = [], []
shopee_base_url_list =[]

In [3]:
#========================
#Generate all related URL 
#=======================
main_url="https://shopee.com.my/search?keyword=ahmad%20jasmine%20green%20tea%20(%2025%20tea%20bags)&page="
for i in range(1):
    temp_url=main_url+str(i)
    shopee_base_url_list.append(temp_url)
print(shopee_base_url_list)

['https://shopee.com.my/search?keyword=ahmad%20jasmine%20green%20tea%20(%2025%20tea%20bags)&page=0', 'https://shopee.com.my/search?keyword=ahmad%20jasmine%20green%20tea%20(%2025%20tea%20bags)&page=1', 'https://shopee.com.my/search?keyword=ahmad%20jasmine%20green%20tea%20(%2025%20tea%20bags)&page=2']


In [4]:
#===============================
#Define Browser
#===============================
# create object for chrome options
chrome_options = Options()
# set chrome driver options to disable any popup's from the website
# to find local path for chrome profile, open chrome browser
# and in the address bar type, "chrome://version"
chrome_options.add_argument('disable-notifications')
chrome_options.add_argument('--disable-infobars')
chrome_options.add_argument('start-maximized')
chrome_options.add_argument('--profile-directory=Default')
chrome_options.add_argument('--user-data-dir=C:/Temp/ChromeProfile')
#chrome_options.add_argument('user-data-dir=C:\\Users\\username\\AppData\\Local\\Google\\Chrome\\User Data\\Default')

# To disable the message, "Chrome is being controlled by automated test software"
chrome_options.add_argument("disable-infobars")
# Pass the argument 1 to allow and 2 to block
chrome_options.add_experimental_option("prefs", { 
    "profile.default_content_setting_values.notifications": 2
    })




In [5]:
#=======================================
#Function-fetch data from specific URL
#=======================================
def fetchData(browser,url):
    while True:
        try:
            base_url = url
            browser.get(base_url)
            delay = 5 #secods
            
            WebDriverWait(browser, delay)
            print ("Page is ready")
            sleep(5)
            html = browser.execute_script("return document.getElementsByTagName('html')[0].innerHTML")
            #print(html)
            soup = BeautifulSoup(html, "html.parser")

            # find_all() returns an array of elements. 
            # We have to go through all of them and select that one you are need. And than call get_text()
            for item_n in soup.find_all('div', class_='_10Wbs- _5SSWfi UjjMrh'):
                print(item_n.get_text())
                item_name.append(item_n.text)

            # find the price of items
            for item_c_outter in soup.find_all('div', class_='zp9xm9 xSxKlK _1heB4J'):
                print(item_c_outter.text)
                item_cost.append(item_c_outter.text)

            # find total number of items sold/month
            for items_s in soup.find_all('div',class_ = '_2VIlt8'):
                print(items_s.get_text())
                items_sold.append(items_s.text)

            # find item location
            for il in soup.find_all('div', class_ = '_1w5FgK'):
                print(il.get_text())
                item_loc.append(il.text)

            break # it will break from the loop once the specific element will be present. 
    
        except TimeoutException:
            print ("Loading took too much time!-Try again")

In [6]:
# invoke the webdriver
browser = webdriver.Chrome(executable_path ="D:\\chromedriver_win32\\chromedriver.exe",
            options = chrome_options)

for i in range(len(shopee_base_url_list)):
    fetchData(browser,shopee_base_url_list[i])
    

Page is ready
Ahmad Tea - Green Tea / Jasmine Green Tea / Earl Grey (100 Tea Bags) / Detox (20Tb)
Ahmad Green Tea / Jasmine Green Tea / Earl Grey (25 Teabags x 1 Box)
Ahmad Tea Green Tea / Detox & Assorted Flavour
TEATICS GUAVA LEAF GREEN TEA 番石榴叶茶 [GOLD PREMIUM QUALITY]- TEH DAUN JAMBU BATU BUATAN MALAYSIA 45 X2g
AHMAD TEA LONDON (Green Tea/Peach & Passion Fruit/Jasmine Green Tea/Detox/Lemon & Ginger)/Teh Hijau/Teh Hitam/ Black Tea
Ahmad Tea Jasmine Green Tea (25 Teabags) Halal Certified
Ahmad Tea Jasmine Green Tea 2g x 25 tea bags
*HALAL*Ahmad Tea Green Tea / Jasmine Green Tea / Earl Grey / English Breakfast - 25 Tea Bags
Ahmad Tea - Jasmine Green Tea (25's x 2g)
AHMAD TEA JASMINE GREEN TEA  (100 /25 Tea Bags)
Ahmad Tea Jasmine Green Tea - 25 teabags
Ahmad Tea Lonton Green Tea / Jasmine Green Tea (25 tea bags)
[Ready Stock] AHMAD TEA Jasmine Green Tea Teh Hijau Jasmin Bunga Melur Great Taste Kaya Antioksidan 25 Teabags
Ahmad Green Tea / Jasmine Green Tea / Earl Grey (25 Teabags x 1 B

# Data Preprocessing

In [7]:
# Data Preprocessing - Prices
#item_cost =['RM13.40-RM15.00','RM17.00']
for i in range(len(item_cost)):
    query1 = item_cost[i]
    if(query1.find('-')!=-1):
        #Remove parts before '-'
         #If the data is 'RM13.40-RM15.00', select the highest value,RM15
        query1 = query1[query1.index('-')+1:]
    #Remove parts before'M' 
    query1 = query1[query1.index('M')+1:]
    item_cost[i]=float(query1)
print(item_cost)

[17.45, 5.3, 11.8, 45.0, 12.9, 5.4, 5.55, 5.74, 5.83, 22.99, 5.0, 5.99, 5.69, 5.3, 25.0]


In [8]:
#Data Preprocessing - Units sold
#items_sold=['167 sold','1.6k sold']
for i in range(len(items_sold)):
    query1=items_sold[i]
    #Remove Space
    query1.replace(" ", "")
    #Remove parts after 's' 
    #'-1' is for remove the white space
    query1 = query1[:query1.index('s')-1]
    if(query1.find('k')!=-1):
        #Turn k into 1000
        query1= query1[:query1.index('k')]
        sold_number=float(query1)*1000
    else:
        sold_number=float(query1)
    items_sold[i]=sold_number
print(items_sold)

[11200.0, 5900.0, 1400.0, 4300.0, 128.0, 2300.0, 4.0, 1700.0, 716.0, 101.0, 250.0, 13.0, 33.0, 5900.0, 50.0]


In [9]:
index_list =[]
index_list=[i for i in range(len(item_name))]

data ={'Item Name':item_name,
      'Price':item_cost,
        'Items Sold':items_sold,
      'Location':item_loc}
print(len(item_name))
print(len(item_cost))
print(len(item_loc))
print(len(items_sold))
create_df = pd.DataFrame(data,index=index_list)

15
15
15
15


# Convert to Excel File

In [10]:
create_df.head()

,Item Name,Price,Items Sold,Location
0,Ahmad Tea - Green Tea / Jasmine Green Tea / Ea...,17.45,11200.0,Perak
1,Ahmad Green Tea / Jasmine Green Tea / Earl Gre...,5.30,5900.0,Perak
2,Ahmad Tea Green Tea / Detox & Assorted Flavour,11.80,1400.0,Perak
3,TEATICS GUAVA LEAF GREEN TEA 番石榴叶茶 [GOLD PREMI...,45.00,4300.0,Kedah
4,AHMAD TEA LONDON (Green Tea/Peach & Passion Fr...,12.90,128.0,Johor


In [11]:
create_df.to_excel("Ahmad Jasmine Green Tea-25 tea bags.xlsx",sheet_name='Page 1')  